In [ ]:
import os 
import glob
import numpy as np 
import pandas as pd

os.chdir('/mnt/BioAdHoc/Groups/vd-ay/hichip-db-loop-calling/')
samplesheet_dir = 'results/samplesheets/chipseq/'
batch = 'batch2'

## Load the GSM Data

In [2]:
fn = os.path.join(samplesheet_dir, 'ChIP-seq-Tracker-Batch2-8.16.22-10.23.23-Step0.HiChIP-to-ChIP-seq-Map.tsv')
data = pd.read_table(fn, skiprows=1)
data = data.loc[data['Has ChIP-seq?'] == 'Yes']

In [3]:
data = pd.read_table(fn, skiprows=1)

data = data.loc[data['Has ChIP-seq?'] == 'Yes']

In [4]:
chipseq_gse_lists = data['GSM ID(s)'].dropna().tolist() + data['Control GSM(s)'].dropna().tolist()
chipseq_gsms = []
for gse_list in chipseq_gse_lists:
    gsm_ids = [x.strip() for x in gse_list.split(',')]
    chipseq_gsms.extend(gsm_ids)
    
chipseq_gsms = sorted(set(chipseq_gsms))

In [5]:
data = data.loc[data['Has ChIP-seq?'] == 'Yes']

Query the below within the SRA Run Selector: https://www.ncbi.nlm.nih.gov/Traces/study/

In [6]:
','.join(chipseq_gsms)

'GSM4952699,GSM4952700,GSM4952701,GSM4952702,GSM4952703,GSM4952704,GSM4952705,GSM4952706,GSM4952707,GSM4952708,GSM4952709,GSM4952710,GSM5034230,GSM5034231,GSM5513456,GSM5625199,GSM5625200,GSM5625201,GSM5625202,GSM5625207,GSM5625208,GSM5625209,GSM5625210,GSM5904685,GSM5904686,GSM5904687,GSM5904688,GSM5904689,GSM5904690,GSM5904691,GSM5904692,GSM5904693,GSM5904694,GSM5904695,GSM5904696,GSM6016304,GSM6016305,GSM6016306,GSM6016307,GSM6016308,GSM6016309,GSM6016310,GSM6016311,GSM6016312,GSM6016313,GSM6016314,GSM6016315,GSM6226246,GSM6226256,GSM6226257,GSM6514267,GSM6514268,GSM6514269,GSM6514270,GSM6514271,GSM6514272,GSM6568001,GSM6568006,GSM6568007,GSM6568008,GSM6568016,GSM6568017,GSM6568227,GSM6568228,GSM6568229,GSM6568230,GSM6568231,GSM6568232,GSM6568233,GSM6568234,GSM6568237,GSM6568238,GSM6568239,GSM6585290,GSM6585291,GSM6614089,GSM6614090,GSM6614091,GSM6620326,GSM6620327,GSM6620328,GSM6620329,GSM6620330,GSM6620331,GSM6620332,GSM6620333,GSM6715172,GSM6715173,GSM6715174,GSM6715175,GSM671517

In [7]:
len(chipseq_gsms)

125

## Parse the Queried Data

In [8]:
# load the data
sra_table_fn = os.path.join(samplesheet_dir, 'chipseq.{}.SraRunTable.txt'.format(batch))
gsm_to_srr_data = pd.read_table(sra_table_fn, sep=',')
gsm_to_srr_data = gsm_to_srr_data[['Sample Name', 'Run']]
gsm_to_srr_data.columns = ['gsm_id', 'srr_id']
gsm_to_srr_data.sort_values('gsm_id', inplace=True)

In [9]:
# manually adding missing samples
adding_missing_entries = [['GSM7336681', 'SRR16538542'],
                          ['GSM7336682', 'SRR16538550'],
                          ['GSM7336684', 'SRR16538541'],
                          ['GSM7336685', 'SRR16538549'],
]
adding_missing_entries = pd.DataFrame(adding_missing_entries, columns=gsm_to_srr_data.columns.tolist())
gsm_to_srr_data = pd.concat([gsm_to_srr_data, adding_missing_entries], axis=0)

## Check for Discrepancies

In [10]:
original_gses = set(chipseq_gsms)

In [11]:
queried_gses = set(gsm_to_srr_data.gsm_id.unique())

In [12]:
len(queried_gses)

125

In [13]:
original_gses.difference(queried_gses)

set()

Not sure how I got these extra stragglers 

In [14]:
queried_gses.difference(original_gses)

set()

In [15]:
gsm_to_srr_data.gsm_id.value_counts()

GSM4952699    3
GSM4952706    3
GSM4952700    3
GSM4952710    3
GSM4952709    3
             ..
GSM6016313    1
GSM6016312    1
GSM6016311    1
GSM6016310    1
GSM7336685    1
Name: gsm_id, Length: 125, dtype: int64

## Create the Samplesheet for Downloading

In [16]:
# get unique SRRs 

In [17]:
srr_ids = sorted(gsm_to_srr_data.srr_id.unique())

In [18]:
srr_ids_fn = os.path.join(samplesheet_dir, 'chipseq.srr_ids.{}.txt'.format(batch))
with open(srr_ids_fn, 'w') as fw:
    fw.write('\n'.join(srr_ids))

## Create the Samplesheet for Concatination

In [19]:
# get Combinations of SRRs that should be concatinated
gsm_srr_combos_data = []

# main GSMs
for gsm_ids_combo in data['GSM ID(s)'].tolist():
    gsm_ids = gsm_ids_combo.replace(' ', '').split(',')
    gsm_combo_reps = [gsm_ids_combo] * len(gsm_ids)
    split_data = list(zip(gsm_combo_reps, gsm_ids))
    gsm_srr_combos_data.extend(split_data)    
    
# control GSMs
for gsm_ids_combo in data['Control GSM(s)'].tolist():
    
    if type(gsm_ids_combo) == str:
        gsm_ids = gsm_ids_combo.replace(' ', '').split(',')
        gsm_combo_reps = [gsm_ids_combo] * len(gsm_ids)
        split_data = list(zip(gsm_combo_reps, gsm_ids))
    else:
        split_data = [[np.nan, np.nan]]
    gsm_srr_combos_data.extend(split_data)    

In [20]:
gsm_srr_combos_data = pd.DataFrame(gsm_srr_combos_data, columns=['GSM ID(s)', 'split_gsm_ids']).drop_duplicates()
gsm_srr_combos_data = gsm_srr_combos_data.merge(gsm_to_srr_data, left_on='split_gsm_ids', right_on='gsm_id', how='outer', indicator=True)
gsm_srr_combos_data = gsm_srr_combos_data.loc[~gsm_srr_combos_data.gsm_id.isna()]

In [21]:
gsm_srr_combos_data_agg = gsm_srr_combos_data.groupby('GSM ID(s)').agg({'srr_id': '-'.join})
srr_combos = sorted(gsm_srr_combos_data_agg.srr_id.unique())

In [22]:
srr_combos_fn = os.path.join(samplesheet_dir, 'chipseq.srr_id.combos.{}.txt'.format(batch))
with open(srr_combos_fn, 'w') as fw:
    fw.write('\n'.join(srr_combos))

In [23]:
len(srr_combos)

81

## Create the Samplesheet for Alignment of Control SRRs

#### Generate a samplesheet that contains the organism

In [24]:
# extracting gsms
df1 = data[['GSM ID(s)', 'Organism']].rename(columns={'GSM ID(s)': 'gsm_ids'})
df1['sample_type'] = 'main'
df2 = data[['Control GSM(s)', 'Organism']].rename(columns={'Control GSM(s)': 'gsm_ids'})
df2['sample_type'] = 'control'

# concating the gsm dfs
gsm_to_organism = pd.concat([df1, df2], axis=0).dropna().drop_duplicates()

# adding srr information
gsm_to_organism = gsm_to_organism.merge(gsm_srr_combos_data_agg, left_on='gsm_ids', right_index=True)

# extract control only
gsm_to_organism_control = gsm_to_organism.loc[gsm_to_organism.sample_type == 'control']

# adding reference genome information
human_subset = gsm_to_organism_control.loc[gsm_to_organism_control['Organism'] == 'Homo Sapiens']
mouse_subset = gsm_to_organism_control.loc[gsm_to_organism_control['Organism'] == 'Mus Musculus']

grch38_subset = human_subset.copy().sort_values(['srr_id'])
grch38_subset.loc[:, 'ref_genome'] = 'grch38'

t2t_subset = human_subset.copy().sort_values(['srr_id'])
t2t_subset.loc[:, 'ref_genome'] = 't2t'

mm10_subset = mouse_subset.copy().sort_values(['srr_id'])
mouse_subset.loc[:, 'ref_genome'] = 'mm10'

# concating all the subsets for a final samplesheet
gsm_to_organism_control = pd.concat([grch38_subset, mouse_subset, t2t_subset])

/tmp/ipykernel_79522/1270239004.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mouse_subset.loc[:, 'ref_genome'] = 'mm10'


In [25]:
# extract srr and ref_genome
gsm_to_organism_control = gsm_to_organism_control[['srr_id', 'ref_genome']]

In [26]:
samplesheet_path = os.path.join(samplesheet_dir, 'chipseq.srr_ids_to_ref_genome.combos.{}.txt'.format(batch))
gsm_to_organism_control.to_csv(samplesheet_path, sep='\t', header=False, index=False)

## Create the Samplesheet for Running ChIPLine

In [27]:
# process only uniqe GSM combos
chipline_ss = data[['Sample Name', 'GSE ID For ChIP-seq Data', 'Organism', 'Pulldown', 'GSM ID(s)', 'Control GSM(s)']]
chipline_ss = chipline_ss.drop_duplicates(subset=['GSM ID(s)'])

In [28]:
# add the srr information
chipline_ss = chipline_ss.merge(gsm_srr_combos_data_agg, left_on='GSM ID(s)', right_index=True, how='left')
chipline_ss.rename(columns={'srr_id': 'Main SRR(s)'}, inplace=True)
chipline_ss = chipline_ss.merge(gsm_srr_combos_data_agg, left_on='Control GSM(s)', right_index=True, how='left')
chipline_ss.rename(columns={'srr_id': 'Control SRR(s)'}, inplace=True)

In [29]:
# add organism infor
chipline_ss.loc[:, 'Organism'] = chipline_ss.loc[:, 'Organism'].str.replace('Homo Sapiens', 'Homo_Sapiens')
chipline_ss.loc[:, 'Organism'] = chipline_ss.loc[:, 'Organism'].str.replace('Mus Musculus', 'Mus_Musculus')

In [30]:
# adding the biorep information 
chipline_ss_tmp_list = []
sort_cols = ['GSE ID For ChIP-seq Data', 'Sample Name', 'Organism', 'Pulldown', 'GSM ID(s)', 'Control GSM(s)']
chipline_ss = chipline_ss.sort_values(sort_cols)
for grp, grp_df in chipline_ss.groupby(['GSE ID For ChIP-seq Data', 'Sample Name', 'Organism', 'Pulldown']):
    grp_df.loc[:, 'Rep No.'] = ['b{}'.format(x) for x in range(1, grp_df.shape[0] + 1)]
    chipline_ss_tmp_list.append(grp_df)
chipline_ss = pd.concat(chipline_ss_tmp_list)

In [31]:
# add the standard sample name 
def create_std_sample_name(sr):
    name = '{}.{}.{}.{}.{}'.format(sr['Sample Name'], sr['GSE ID For ChIP-seq Data'],
                                   sr['Organism'], sr['Pulldown'], sr['Rep No.'])
    return(name)
chipline_ss.loc[:, 'chipseq_std_sample_name'] = chipline_ss.apply(create_std_sample_name, axis=1)

In [32]:
chipline_ss

,Sample Name,GSE ID For ChIP-seq Data,Organism,Pulldown,GSM ID(s),Control GSM(s),Main SRR(s),Control SRR(s),Rep No.,chipseq_std_sample_name
0,VCP65,GSE162609,Homo_Sapiens,H3K27ac,"GSM4952699, GSM4952700, GSM4952701","GSM4952702, GSM4952703, GSM4952704",SRR13178581-SRR13178579-SRR13178580-SRR1317858...,SRR13178588-SRR13178590-SRR13178589-SRR1317859...,b1,VCP65.GSE162609.Homo_Sapiens.H3K27ac.b1
1,VCP67,GSE162609,Homo_Sapiens,H3K27ac,"GSM4952705, GSM4952706, GSM4952707","GSM4952708, GSM4952709, GSM4952710",SRR13178599-SRR13178598-SRR13178597-SRR1317860...,SRR13178606-SRR13178607-SRR13178608-SRR1317860...,b1,VCP67.GSE162609.Homo_Sapiens.H3K27ac.b1
2,HCT116-NUP93-mAC,GSE165463,Homo_Sapiens,H3K27ac,GSM5034230,NaN,SRR13523286,NaN,b1,HCT116-NUP93-mAC.GSE165463.Homo_Sapiens.H3K27a...
4,HCT116-NUP93-mAC-IAA,GSE165463,Homo_Sapiens,H3K27ac,GSM5034231,NaN,SRR13523287,NaN,b1,HCT116-NUP93-mAC-IAA.GSE165463.Homo_Sapiens.H3...
39,YAP-M-tumor,GSE181867,Mus_Musculus,H3K27ac,GSM5513456,NaN,SRR15414689,NaN,b1,YAP-M-tumor.GSE181867.Mus_Musculus.H3K27ac.b1
62,BMDC,GSE185880,Mus_Musculus,H3K27ac,"GSM5625207, GSM6620326, GSM6620327",GSM5625199,SRR16323887-SRR21839429-SRR21839428,SRR16323879,b1,BMDC.GSE185880.Mus_Musculus.H3K27ac.b1
6,BMDC-CTCF_cKO,GSE185880,Mus_Musculus,H3K27ac,GSM5625209,GSM5625201,SRR16323889,SRR16323881,b1,BMDC-CTCF_cKO.GSE185880.Mus_Musculus.H3K27ac.b1
8,BMDC-CTCF_cKO-LPS,GSE185880,Mus_Musculus,H3K27ac,GSM5625210,GSM5625202,SRR16323890,SRR16323882,b1,BMDC-CTCF_cKO-LPS.GSE185880.Mus_Musculus.H3K27...
64,BMDC-JSH,GSE185880,Mus_Musculus,H3K27ac,"GSM6620328, GSM6620329",NaN,SRR21839427-SRR21839426,NaN,b1,BMDC-JSH.GSE185880.Mus_Musculus.H3K27ac.b1
66,BMDC-JSH-JSH-LPS,GSE185880,Mus_Musculus,H3K27ac,"GSM6620330, GSM6620331",NaN,SRR21839425-SRR21839424,NaN,b1,BMDC-JSH-JSH-LPS.GSE185880.Mus_Musculus.H3K27a...


In [33]:
# organize the reference genome of samples
chipline_ss.loc[:, 'Ref Genome'] = chipline_ss['Organism'].map({'Homo_Sapiens': 'grch38',
                                                                 'Mus_Musculus': 'mm10'})
# add T2T samples
chipline_ss_grps = chipline_ss.groupby('Ref Genome')
t2t_chipline_ss = chipline_ss_grps.get_group('grch38').copy(deep=True)
t2t_chipline_ss.loc[:, 'Ref Genome'] = 't2t'

In [34]:
# generate teh final chipline samplesheet
chipline_ss = pd.concat([chipline_ss, t2t_chipline_ss])

In [35]:
# organize the columns 
chipline_ss = chipline_ss[['chipseq_std_sample_name', 'Main SRR(s)', 'Control SRR(s)', 'Ref Genome',
                           'Sample Name', 'GSE ID For ChIP-seq Data',
                            'Organism', 'Pulldown', 'Rep No.', 'GSM ID(s)', 'Control GSM(s)']]

In [36]:
# finalize and save
chipline_ss.fillna('N/A', inplace=True)
chipline_ss_fn = os.path.join(samplesheet_dir, 'chipseq.chipline.{}.txt'.format(batch))
chipline_ss.to_csv(chipline_ss_fn, sep='\t', index=False, header=False)

In [46]:
chipline_ss.shape

(96, 11)

## Add the ChIP-seq file that finally maps between HiChIP and ChIP-seq

In [38]:
mapper = data.copy()
mapper.rename(columns={'Replicate Serial No': 'Rep No.'}, inplace=True)
mapper.loc[:, 'Organism'] = mapper.loc[:, 'Organism'].str.replace(' ' , '_') 
mapper.loc[:, 'Rep No.'] = ['b{}'.format(x) for x in mapper.loc[:, 'Rep No.'].astype(int)]
mapper.loc[:, 'chipseq_std_sample_name'] = mapper.apply(create_std_sample_name, axis=1)

def get_chipseq_path(std_sample_name):
    
    res_dir = 'results/peaks/chipline_v2/'
    sample_tmpl = os.path.join(res_dir, '{0}/MACS2_Ext_*/{0}.macs2_peaks.narrowPeak_Q0.01filt'.format(std_sample_name))
    print(sample_tmpl)
    peak_fns = glob.glob(sample_tmpl)
    
    if len(peak_fns) == 1:
        return(peak_fns[0])
    elif len(peak_fns) > 1:
        return('Found multiple files')
    else:
        return('Could not find a matching file')

mapper.loc[:, 'chipseq_path'] = mapper.loc[:, 'chipseq_std_sample_name'].apply(get_chipseq_path)
fn = os.path.join(samplesheet_dir, 'chipseq.tracker.hichip_to_chipseq.paths_mapped.{}.tsv'.format(batch))
mapper.to_csv(fn, sep='\t', index=False, header=True)